# Unit test Generator

Create unit tests on the Python code

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
import anthropic
import ollama
import gradio as gr

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
client = genai.Client()


OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-sonnet-4-20250514"
GEMINI_MODEL = 'gemini-2.5-flash'
LLAMA_MODEL = "llama3.2"


In [ ]:
system_message = """
You are an effective programming assistant specialized to generate Python code based on the inputs.
Respond only with Python code; use comments sparingly and do not provide any explanation other than occasional comments.
Do not include Markdown formatting (```), language tags (python), or extra text \n.
"""

In [ ]:
def user_prompt_for_unit_test(python):
    user_prompt = f"""
    Consider the following Python code: \n\n
        {python} \n\n

        Generate a unit test around this code and it alongside with the Python code. \n
        Response rule: in your response do not include Markdown formatting (```), language tags (python), or extra text.

    """
    return user_prompt

In [ ]:
def messages_for_unit_test(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for_unit_test(python)}
    ]

In [ ]:
python_hard = """

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum
"""

In [ ]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for_unit_test(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply

In [ ]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for_unit_test(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply

In [ ]:
def stream_gemini(python):
    response = client.models.generate_content_stream(
    model=GEMINI_MODEL,
    config=types.GenerateContentConfig(
        system_instruction=system_message),
    contents=user_prompt_for_unit_test(python)
    )

    reply = ""
    for chunk in response:
        fragment = chunk.text or ""
        reply += fragment
        yield reply



In [ ]:
def stream_llama_local(python):
    stream = ollama.chat(
        model='llama3.2',
        messages=messages_for_unit_test(python),
        stream=True,
    )

    reply = ""
    # Iterate through the streamed chunks and print the content
    for chunk in stream:
        #print(chunk['message']['content'], end='', flush=True)
        if 'content' in chunk['message']:
            fragment = chunk['message']['content']
            reply += fragment
            yield reply


In [ ]:
def generate_unit_test(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    elif model=="Llama":
        result = stream_llama_local(python)

    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        unit_test = gr.Textbox(label="Unit test", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini", "Llama"], label="Select model", value="GPT")
        generate_ut = gr.Button("Generate Unit tests")

    generate_ut.click(generate_unit_test, inputs=[python, model], outputs=[unit_test])

ui.launch(inbrowser=True)